In [31]:
import pandas as pd
import numpy as np
from fredapi import Fred
import yfinance as yf
import json
import requests
from dotenv import load_dotenv
import os

load_dotenv()

True

### Setting parameters

In [32]:
# setting the api keys
fred_api_key = os.getenv('FRED_API_KEY')
bls_api_key = os.getenv('BLS_API_KEY')

fred=Fred(api_key=fred_api_key)

# date range of the datasets
start_date = '1990-01-01'
end_date = '2025-01-01'

In [33]:
# table of datasets info
df_sets = pd.read_csv('Data_info.csv', delimiter=';')

# general info of S&P500 composition and BLS endpoint
sp500_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
data_table = pd.read_html(sp500_url)

BLS_ENDPOINT = "https://api.bls.gov/publicAPI/v2/timeseries/data/"

# Creating the object with S&P500 ticker
sp500_index = yf.Ticker('^GSPC')

### Extracting data

In [34]:
# S&P500 index
historical_index_data = sp500_index.history(start=start_date, end=end_date)

df_SP500 = historical_index_data[['Close']].reset_index()

df_SP500['Date'] = pd.to_datetime(df_SP500['Date']).dt.date


In [35]:
# Extrating FRED series and saving it in the path
# Create a folder on the actual path to save the files
folder_name = "Datasets"

for index, row in df_sets.iterrows():
    series_id = row['Series_Id']
    description = row['File_name']
    series_name = row['Series_Name']

    # Get series data from FRED and save dataframe in a CSV
    basedf_fred = fred.get_series(series_id=series_id)
    df_fred = pd.DataFrame(basedf_fred, columns=[series_id])
    df_fred = df_fred.rename_axis('Date').reset_index()
    full_date_range = pd.date_range(start=start_date, end=end_date, freq='D')
    df_fred = df_fred.set_index('Date').reindex(full_date_range).rename_axis('Date').reset_index()

    df_fred[series_id] = df_fred[series_id].bfill()
    df_fred[series_id] = df_fred[series_id].ffill()

    file_name = os.path.join(folder_name, f"{description}.csv")
    df_fred.to_csv(file_name, index=False)
    print(f"Saved: {file_name}")

Saved: Datasets\Bank_Credit_All_Commercial_Banks.csv
Saved: Datasets\10Year_Real_Interest_Rate.csv
Saved: Datasets\Commercial_Real_Estate_Prices_for_United_States.csv
Saved: Datasets\Consumer_Loans_Credit_Cards_and_Other_Revolving_Plans_All_Commercial_Banks.csv
Saved: Datasets\Market_Yield_on_US_Treasury_Securities_at_10Year_Constant_Maturity_Quoted_on_an_Investment_Basis.csv
Saved: Datasets\Consumer_Price_Index_for_All_Urban_Consumers.csv
Saved: Datasets\Continued_Claims_Insured_Unemployment.csv
Saved: Datasets\Delinquency_Rate_on_Credit_Card_Loans_All_Commercial_Banks.csv
Saved: Datasets\Federal_Funds_Effective_Rate.csv
Saved: Datasets\Gross_Domestic_Product.csv
Saved: Datasets\Households_Owners_Equity_in_Real_Estate_Level.csv
Saved: Datasets\M2.csv
Saved: Datasets\Median_Consumer_Price_Index.csv
Saved: Datasets\NASDAQ_Composite_Index.csv
Saved: Datasets\Personal_Saving_Rate.csv
Saved: Datasets\Real_Estate_Loans_All_Commercial_Banks.csv
Saved: Datasets\Real_Gross_Domestic_Product.csv

In [36]:

# One list for tickers and a dataframe to extract ticker information in the future
tickers = data_table[0]['Symbol'].tolist()
Sp500_companies = data_table[0][['Symbol', 'Security', 'GICS Sector']]

# Making adjustments on the dataset
for i in range(len(tickers)):
    if tickers[i] == 'BRK.B':
        tickers[i] = 'BRK-B'
    elif tickers[i] == 'BF.B':  
        tickers[i] = 'BF-B'

# Adjustments in some ticker codes
for index, row in Sp500_companies.iterrows():
    if row['Symbol'] == 'BRK.B':
        Sp500_companies.at[index, 'Symbol'] = 'BRK-B'
    elif row['Symbol'] == 'BF.B':
        Sp500_companies.at[index, 'Symbol'] = 'BF-B'

ticker_prices = yf.download(tickers, start=start_date, end=end_date)['Close']


[*********************100%***********************]  503 of 503 completed


In [44]:
# DXY dataset
dxy_df = yf.download('DX-Y.NYB', start=start_date, end=end_date)['Close']

[*********************100%***********************]  1 of 1 completed


In [38]:
# BLS series, in this case is inflation, but could extract more than one serie
def bls_series(series, start, end, api_key, end_point):

    headers = {'Content-type': 'application/json'}
    payload = {
        'seriesid': series, 
        'startyear': start, 
        'endyear': end,
        'registrationkey': api_key        
    }
    data = json.dumps(payload)
    response = requests.post(end_point, data=data, headers=headers)
    json_data = json.loads(response.text)
    dfs = []

    for serie in json_data['Results']['series']:
        df_initial = pd.DataFrame(serie)
        series_col = df_initial['seriesID'][0]

        for i in range(0, len(df_initial) -1):
            df_row = pd.DataFrame(df_initial['data'][i])
            df_row['seriesId'] = series_col

            if 'code' not in str(df_row['footnotes']):
                df_row['footnotes'] = ''
            else:
                df_row['footnotes'] = str(df_row['footnotes']).split("'code': '", 1)[1][:1]

            dfs.append(df_row)

    df = pd.concat(dfs, ignore_index=True)
    return df

series = ['CUUR0000SA0']
bls_data1 = bls_series(series, 1987, 2006, bls_api_key, BLS_ENDPOINT)
bls_data2 = bls_series(series, 2007, 2025, bls_api_key, BLS_ENDPOINT)
full_cpi = pd.concat([bls_data1, bls_data2], ignore_index=True )

In [39]:
# Create the new column 'date' in YYYY-MM-DD format
full_cpi['Date'] = pd.to_datetime(full_cpi['year'].astype(str) + 
                                  full_cpi['periodName'], format='%Y%B').dt.strftime('%Y-%m-%d')


In [40]:
#sorting the results by date and creating the dataframe with percent rate of CPI

sorted_full_cpi = full_cpi.sort_values(by='Date')
sorted_full_cpi['Date'] = pd.to_datetime(sorted_full_cpi['Date'])
sorted_full_cpi['value'] = pd.to_numeric(sorted_full_cpi['value'], errors='coerce')
sorted_full_cpi.set_index('Date', inplace=True)
sorted_full_cpi['CPI Rate'] = float('nan')
sorted_full_cpi['CPI Rate'] = sorted_full_cpi['value'].pct_change(periods=12) * 100
sorted_full_cpi.reset_index(inplace=True)

sorted_full_cpi = sorted_full_cpi[['Date', 'CPI Rate']]

In [ ]:
# exporting the data - make sure to have 'Datasets' folder on the same path of this file

df_SP500.to_csv('Datasets/S&P500.csv', index=False)
ticker_prices.to_csv('Datasets/tickers.csv', index=False)
Sp500_companies.to_csv('Datasets/Sp500_companies.csv', index=False)
sorted_full_cpi.to_csv('Datasets/CPI.csv', index=False)
dxy_df.to_csv('Datasets/DXY.csv')

print(f"All files are saved in the folder: {folder_name}")

All files are saved in the folder: Datasets
